<a href="https://colab.research.google.com/github/Arpith01/directional_stock_prediction/blob/master/notebooks/models/sentiment_ngram_features_SVM_LR_NB_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack
import pandas as pd
import numpy as np
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm
import scipy as sp
import pickle

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
amazon_labelled = pd.read_csv('/content/drive/MyDrive/new_data_ARS/Label42223.csv')
apple_labelled = pd.read_csv('/content/drive/MyDrive/new_data_ARS/Label377189.csv')
# project_data1.shape

amazon_labelled = amazon_labelled[amazon_labelled.text.notnull()]
apple_labelled = apple_labelled[apple_labelled.text.notnull()]
project_data = pd.concat([amazon_labelled, apple_labelled])
project_data.shape


(419414, 3)

In [ ]:
print("Total data points in train data", project_data.shape)
print("The attributes of data :", project_data.columns.values)

Total data points in train data (419414, 3)
The attributes of data : ['Unnamed: 0' 'text' 'label']


In [ ]:
project_data.head(10)
# project_data.drop(['Unnamed: 0'], axis=1, inplace=True)

,Unnamed: 0,text,label
0,0,amazon soon resum sell appl tv nasdaq aapl goo...,1
1,1,nyse initi oper checkout line cashier similar ...,1
2,2,nt fit profil fang stock facebook fb amazoncom...,0
3,3,appl amazon amzn microsoft alreadi cloudcomput...,0
4,4,roku stream stick plu offer k hdr play googl a...,0
5,5,stick plu stream k hdr content get access goog...,0
6,6,amazon fire tv play k hdr content interact sma...,0
7,7,roku stream stick plu favorit devic didnt exis...,0
8,8,latest iter amazon amzn cut fire tv bodi diamo...,0
9,9,fire tv also come amazon alexa voic assist built,0


In [ ]:
y = project_data['label'].values
# project_data.drop(['label'], axis=1, inplace=True)
print(project_data.head(5))
X = project_data.text.values

   Unnamed: 0                                               text  label
0           0  amazon soon resum sell appl tv nasdaq aapl goo...      1
1           1  nyse initi oper checkout line cashier similar ...      1
2           2  nt fit profil fang stock facebook fb amazoncom...      0
3           3  appl amazon amzn microsoft alreadi cloudcomput...      0
4           4  roku stream stick plu offer k hdr play googl a...      0


In [ ]:
print(y)
print(X)

[1 1 0 ... 1 1 0]
['amazon soon resum sell appl tv nasdaq aapl googl chromecast goog pull ecommerc platform two year ago support prime video amzn '
 'nyse initi oper checkout line cashier similar fashion futurist amazon go nasdaq amzn store announc year ago yet open public '
 'nt fit profil fang stock facebook fb amazoncom amzn netflix nflx alphabetgoogl googl drug intern '
 ...
 'earlier two congressmen sent letter appl ceo tim cook question includ learn issu impact flaw undisclos bug '
 'busi insid appl apolog bug last friday fix group facetim secur server issu softwar updat reenabl featur user next '
 'take secur product extrem serious commit continu earn trust appl custom place us ']


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y)
# X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.33, stratify=y_train)

In [ ]:
# print(X_train)
# type(X_train)
# X_train.shape
# X.shape
X[0]

'amazon soon resum sell appl tv nasdaq aapl googl chromecast goog pull ecommerc platform two year ago support prime video amzn '

### *Please Copy the cells below based on your requirement*

# Sentimental Features

#### If you would like to use additional features i.e., neutral and compound scores apart from positive and negative scores for each text use the code below. similarly, Apple data features can also be extracted.

In [ ]:
cls = SentimentIntensityAnalyzer()
# Q = X_train["text"].copy()
Positive_X_train = []
Negative_X_train = []
Neutral_X_train = []
Compound_X_train = []

for i in tqdm(X_train):
    n1= cls.polarity_scores(i)["neg"]
    n2= cls.polarity_scores(i)["pos"]
    n3= cls.polarity_scores(i)["neu"]
    n4= cls.polarity_scores(i)["compound"]
    Positive_X_train.append(n1)
    Negative_X_train.append(n2)
    Neutral_X_train.append(n3)
    Compound_X_train.append(n4)

100%|██████████| 281007/281007 [02:44<00:00, 1710.76it/s]


## Train

In [ ]:
# Q = X_train["text"].copy()

cls = SentimentIntensityAnalyzer()

Positive_X_train_text = []
Negative_X_train_text = []

for i in tqdm(X_train):
    n1= cls.polarity_scores(i)["neg"]
    n2= cls.polarity_scores(i)["pos"]
    Positive_X_train_text.append(n1)
    Negative_X_train_text.append(n2)
    

100%|██████████| 281007/281007 [01:22<00:00, 3392.29it/s]


In [ ]:
print(Positive_X_train_text[1000])
print(Negative_X_train_text[1000])
# print(X_train["text"][1000])

0.0
0.306


## Test

In [ ]:
cls = SentimentIntensityAnalyzer()

Positive_X_test_text = []
Negative_X_test_text = []

for i in tqdm(X_test):
    n1= cls.polarity_scores(i)["neg"]
    n2= cls.polarity_scores(i)["pos"]
    Positive_X_test_text.append(n1)
    Negative_X_test_text.append(n2)
    

100%|██████████| 138407/138407 [00:41<00:00, 3352.23it/s]


In [ ]:
print(Positive_X_test_text[1000])
print(Negative_X_test_text[1000])

0.0
0.524


# N-grams 

### 2-grams

#### Considering the words that appeared in atleast 10 documents and top 4000 features considering dimensionality issues.

In [ ]:
vectorizer = CountVectorizer(ngram_range = (2,2),max_features = 4000, min_df = 10)

X_train_preprocessed_text = X_train
X_test_preprocessed_text = X_test

X_train_2gram_features = vectorizer.fit_transform(X_train_preprocessed_text)


In [ ]:
vectorizer_dict = {"vectorizer" : vectorizer}

In [ ]:
import pickle
outfile = open('vectorizer','wb')
pickle.dump(vectorizer_dict,outfile)
outfile.close()

In [ ]:
X_test_2gram_features = vectorizer.transform(X_test_preprocessed_text)

### 3-grams

In [ ]:
vectorizer = CountVectorizer(ngram_range = (3,3),max_features = 4000, min_df = 10)

X_train_preprocessed_text = X_train
X_test_preprocessed_text = X_test

X_train_3gram_features = vectorizer.fit_transform(X_train_preprocessed_text)
X_test_3gram_features = vectorizer.transform(X_test_preprocessed_text)

### Merging the features

In [ ]:
import numpy as np
Sentiment_X_train_text = np.column_stack((Positive_X_train_text, Negative_X_train_text))
Sentiment_X_test_text = np.column_stack((Positive_X_test_text, Negative_X_test_text))

In [ ]:
Sentiment_X_train_text.shape

(281007, 2)

In [ ]:
X_train_2gram_features_array = X_train_2gram_features.toarray()

In [ ]:
X_train_2gram_features_array.shape


(281007, 4000)

In [ ]:
Xtrain = np.concatenate((Sentiment_X_train_text,X_train_2gram_features_array), axis = 1)
Xtrain.shape

(281007, 4002)

In [ ]:
X_test_2gram_features_array = X_test_2gram_features.toarray()
Xtest = np.concatenate((Sentiment_X_test_text,X_test_2gram_features_array), axis = 1)
Xtest.shape

(138407, 4002)

In [ ]:
y_train.shape

(281007,)

In [ ]:
y_test.shape

(138407,)

## SVM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### "Linear"

In [ ]:
from sklearn.svm import LinearSVC
svclassifier = LinearSVC(loss='hinge', verbose=10, max_iter=100000)

In [ ]:
%%time
svclassifier.fit(Xtrain, y_train)

[LibLinear]Wall time: 3min 44s


LinearSVC(loss='hinge', max_iter=100000, verbose=10)

In [ ]:
y_pred = svclassifier.predict(Xtest)

In [ ]:
filename = 'SVM_LinearSVC_1.sav'
pickle.dump(svclassifier, open(filename, 'wb'))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[90960  6235]
 [17999 23213]]
              precision    recall  f1-score   support

           0       0.83      0.94      0.88     97195
           1       0.79      0.56      0.66     41212

    accuracy                           0.82    138407
   macro avg       0.81      0.75      0.77    138407
weighted avg       0.82      0.82      0.82    138407



In [ ]:
from sklearn.svm import LinearSVC
svclassifier = LinearSVC(loss='squared_hinge', verbose=10, max_iter=100000) 

In [ ]:
%%time
svclassifier.fit(Xtrain, y_train)

[LibLinear]Wall time: 49.3 s


LinearSVC(max_iter=1000000, verbose=10)

In [ ]:
y_pred = svclassifier.predict(Xtest)

In [ ]:
filename = 'SVM_LinearSVC_2.sav'
pickle.dump(svclassifier, open(filename, 'wb'))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[90615  6580]
 [17630 23582]]
              precision    recall  f1-score   support

           0       0.84      0.93      0.88     97195
           1       0.78      0.57      0.66     41212

    accuracy                           0.83    138407
   macro avg       0.81      0.75      0.77    138407
weighted avg       0.82      0.83      0.82    138407



#### Gausian Kernel

In [ ]:
svclassifier = SVC(kernel='rbf') #Gaussian Kernel
svclassifier.fit(Xtrain, y_train)

In [ ]:
y_pred = svclassifier.predict(Xtest)

In [ ]:
filename = 'SVM_Gaussian.sav'
pickle.dump(svclassifier, open(filename, 'wb'))

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

#### Sigmoid kernel

In [ ]:
svclassifier = SVC(kernel='sigmoid')
svclassifier.fit(Xtrain, y_train)

In [ ]:
filename = 'SVM_sigmoid.sav'
pickle.dump(svclassifier, open(filename, 'wb'))

In [ ]:
y_pred = svclassifier.predict(Xtest)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', random_state=0)

In [ ]:
model.fit(Xtrain, y_train)

LogisticRegression(random_state=0, solver='liblinear')

In [ ]:
filename = 'log_reg.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
y_pred_LR = model.predict(Xtest)

In [ ]:
print(confusion_matrix(y_test,y_pred_LR))
print(classification_report(y_test,y_pred_LR))

[[90272  6923]
 [17400 23812]]
              precision    recall  f1-score   support

           0       0.84      0.93      0.88     97195
           1       0.77      0.58      0.66     41212

    accuracy                           0.82    138407
   macro avg       0.81      0.75      0.77    138407
weighted avg       0.82      0.82      0.82    138407



## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(Xtrain, y_train)

GaussianNB()

In [ ]:
y_pred_NB = model.predict(Xtest)

In [ ]:
filename = 'naive_bayes.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
print(confusion_matrix(y_test,y_pred_NB))
print(classification_report(y_test,y_pred_NB))

[[49095 48100]
 [ 2080 39132]]
              precision    recall  f1-score   support

           0       0.96      0.51      0.66     97195
           1       0.45      0.95      0.61     41212

    accuracy                           0.64    138407
   macro avg       0.70      0.73      0.64    138407
weighted avg       0.81      0.64      0.65    138407

